In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [11]:
N=10000. #Change this line adjust the number of rows. 
data=pd.read_csv("HIGGS.csv",nrows=N,header=None)
test_data=pd.read_csv("HIGGS.csv",nrows=500,header=None,skiprows=1000)

In [12]:
def size_it (df):
    total=0
    for i in range(len(data.memory_usage())):
        total=total+data.memory_usage()[i]
    size=total/(2**20)
    print("%.2f Megabytes" % size)
    

In [13]:
size_it(data)

2.29 Megabytes


In [14]:
data.columns=['label','lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt',
             'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt',
             'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 'm_jj',
             'm_jjj', 'm_lv','m_jlv', 'm_bb', 'm_wbb', 'm_wwbb']

In [15]:
test_data.columns=['label','lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt',
             'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt',
             'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 'm_jj',
             'm_jjj', 'm_lv','m_jlv', 'm_bb', 'm_wbb', 'm_wwbb']

In [16]:
data.tail()

,label,lepton pT,lepton eta,lepton phi,missing energy magnitude,missing energy phi,jet 1 pt,jet 1 eta,jet 1 phi,jet 1 b-tag,...,jet 4 eta,jet 4 phi,jet 4 b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
9995,1.0,0.743017,0.390507,-0.431368,0.734279,0.491310,1.748141,1.165475,-1.033300,0.000000,...,-0.340253,1.074148,3.101961,0.768013,1.058611,0.979336,1.028349,0.964556,0.915222,0.816399
9996,0.0,2.778078,-0.498726,-0.236057,0.625503,0.443076,1.353130,0.014828,1.449298,2.173076,...,0.779062,0.496485,0.000000,0.921317,0.814733,1.050062,1.759452,1.297594,1.164259,0.962886
9997,0.0,0.602100,2.008270,-1.646517,0.566484,-1.323110,0.597096,0.511923,0.191420,1.086538,...,2.048285,-0.696127,0.000000,0.711091,0.863039,0.986306,0.672815,0.244809,0.607074,0.588277
9998,0.0,0.771017,-1.224333,0.076987,0.819454,1.574369,0.411866,-0.686255,-1.531129,2.173076,...,2.454702,-0.137882,0.000000,0.847904,1.234889,1.074361,0.675443,0.374609,1.027287,1.185665
9999,0.0,0.545184,-1.011034,0.280067,0.152822,-0.196007,1.035253,-1.189291,-1.137523,2.173076,...,-1.082298,-0.181720,1.550981,1.676911,1.059162,0.983805,1.137410,0.705999,1.035395,0.914397


In [17]:
test_data.tail()

,label,lepton pT,lepton eta,lepton phi,missing energy magnitude,missing energy phi,jet 1 pt,jet 1 eta,jet 1 phi,jet 1 b-tag,...,jet 4 eta,jet 4 phi,jet 4 b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
495,0.0,0.664872,0.741136,-0.440246,0.777536,-1.616220,0.575477,0.489148,1.327335,2.173076,...,1.046398,0.236231,0.000000,0.623314,0.750138,0.987102,0.556310,0.385829,0.490488,0.548002
496,0.0,0.485157,-0.845459,-0.326500,1.232177,1.267600,0.593523,0.879299,1.079530,2.173076,...,1.163826,-0.167293,0.000000,0.855666,0.903943,1.061573,0.763485,1.337028,0.892146,0.896002
497,0.0,1.152957,-0.408147,-1.305832,2.302761,-1.040174,0.946761,-1.214047,0.949806,0.000000,...,-1.972586,0.576392,0.000000,1.415086,0.846983,0.987952,0.883330,1.297286,1.205222,1.544976
498,0.0,0.840195,-0.387694,-1.052816,0.413459,0.273790,0.617891,1.072394,-1.110913,0.000000,...,-0.378562,0.707351,3.101961,0.734939,0.759103,0.980972,0.810319,0.200622,0.812651,0.759065
499,1.0,0.411587,1.151177,0.526980,1.115618,-0.553816,2.421728,-1.471507,-1.297183,2.173076,...,0.095314,-0.104588,0.000000,0.645097,0.947385,0.985642,4.948816,3.734236,3.626050,2.588634


In [18]:
data.iloc[:,0:10].describe()

,label,lepton pT,lepton eta,lepton phi,missing energy magnitude,missing energy phi,jet 1 pt,jet 1 eta,jet 1 phi,jet 1 b-tag
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.529500,0.997924,-0.016681,-0.003486,0.991385,-0.009822,0.992058,-0.001468,0.003751,1.004939
std,0.499154,0.574965,1.002943,1.010838,0.595460,1.004828,0.477408,1.004115,1.015907,1.026965
min,0.000000,0.275063,-2.425236,-1.742508,0.012355,-1.743755,0.159488,-2.941008,-1.741237,0.000000
25%,0.000000,0.596061,-0.744166,-0.872486,0.571250,-0.885740,0.679817,-0.682789,-0.892627,0.000000
50%,1.000000,0.858770,-0.028786,0.000643,0.886284,-0.019940,0.897522,-0.009928,0.020396,1.086538
75%,1.000000,1.248305,0.714839,0.881675,1.294626,0.856703,1.170305,0.680263,0.878984,2.173076
max,1.000000,6.695388,2.429998,1.743236,5.824007,1.742818,7.064657,2.969674,1.741454,2.173076


In [19]:
data.iloc[:,10:20].describe()

,jet 2 pt,jet 2 eta,jet 2 phi,jet 2 b-tag,jet 3 pt,jet 3 eta,jet 3 phi,jet 3 b-tag,jet 4 pt,jet 4 eta
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.987711,-0.020984,-0.000168,0.981078,0.994583,0.013844,0.007515,1.003491,0.979784,0.001222
std,0.489934,1.009462,1.005342,1.045674,0.485817,1.016974,0.997067,1.195342,0.499127,1.005459
min,0.189988,-2.910175,-1.742372,0.000000,0.263608,-2.727842,-1.742069,0.000000,0.365354,-2.496432
25%,0.658223,-0.719728,-0.870734,0.000000,0.655243,-0.689796,-0.845616,0.000000,0.620445,-0.708360
50%,0.885920,-0.019853,-0.004364,0.000000,0.900762,0.017468,0.004898,0.000000,0.866529,-0.001294
75%,1.196902,0.665451,0.860579,2.214872,1.224476,0.706753,0.863212,2.548224,1.205413,0.719931
max,5.192512,2.909324,1.743175,2.214872,6.523279,2.727278,1.741774,2.548224,6.067653,2.495511


In [20]:
types={}
for i in data.columns:
    types[i]=data[i].dtype

In [21]:
types

{'label': dtype('float64'),
 'lepton pT': dtype('float64'),
 'lepton eta': dtype('float64'),
 'lepton phi': dtype('float64'),
 'missing energy magnitude': dtype('float64'),
 'missing energy phi': dtype('float64'),
 'jet 1 pt': dtype('float64'),
 'jet 1 eta': dtype('float64'),
 'jet 1 phi': dtype('float64'),
 'jet 1 b-tag': dtype('float64'),
 'jet 2 pt': dtype('float64'),
 'jet 2 eta': dtype('float64'),
 'jet 2 phi': dtype('float64'),
 'jet 2 b-tag': dtype('float64'),
 'jet 3 pt': dtype('float64'),
 'jet 3 eta': dtype('float64'),
 'jet 3 phi': dtype('float64'),
 'jet 3 b-tag': dtype('float64'),
 'jet 4 pt': dtype('float64'),
 'jet 4 eta': dtype('float64'),
 'jet 4 phi': dtype('float64'),
 'jet 4 b-tag': dtype('float64'),
 'm_jj': dtype('float64'),
 'm_jjj': dtype('float64'),
 'm_lv': dtype('float64'),
 'm_jlv': dtype('float64'),
 'm_bb': dtype('float64'),
 'm_wbb': dtype('float64'),
 'm_wwbb': dtype('float64')}

#### Splitting the data into features and output as well test data.

In [22]:
y = np.array(data.iloc[:,0])
x = np.array(data.iloc[:,1:])

In [23]:
x_test = np.array(test_data.iloc[:,1:])
y_test = np.array(test_data.iloc[:,0])

#### Basic modeling

In [24]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model.add(Activation('sigmoid'))
model.add(Dropout(0.10))
model.add(Dense(50, kernel_initializer='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.10))
model.add(Dense(1, kernel_initializer='uniform')) 
model.add(Activation('sigmoid'))

In [25]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

In [ ]:
model.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model.predict(x_test))

Epoch 1/5
